In [1]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001F79F1BF440> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F79F1D90D0> root_client=<openai.OpenAI object at 0x000001F79E3724B0> root_async_client=<openai.AsyncOpenAI object at 0x000001F79F1BF4A0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [2]:
from langchain_community.document_loaders import TextLoader

loader=TextLoader('speech.txt')
loader

In [3]:
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [4]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter=CharacterTextSplitter(separator="\n\n",chunk_size=100,chunk_overlap=20)
text_splitter.split_documents(text_documents)

Created a chunk of size 470, which is longer than the specified 100
Created a chunk of size 347, which is longer than the specified 100
Created a chunk of size 668, which is longer than the specified 100
Created a chunk of size 982, which is longer than the specified 100
Created a chunk of size 789, which is longer than the specified 100


[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [5]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [6]:
from langchain_community.vectorstores import FAISS

vectorstore=FAISS.from_documents(text_documents,embeddings)
vectorstore

In [7]:
query="It is a distressing and oppressive duty, gentlemen of the Congress,"

result=vectorstore.similarity_search(query)
result[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """

Answer the following question based only on the provided context: 
< context> 
{context} 
</context>


"""
)
prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n\nAnswer the following question based only on the provided context: \n< context> \n{context} \n</context>\n\n\n'), additional_kwargs={})])

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n\nAnswer the following question based only on the provided context: \n< context> \n{context} \n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F79F1BF440>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F79F1D90D0>, root_client=<openai.OpenAI object at 0x000001F79E3724B0>, root_async_client=<openai.AsyncOpenAI object at 0x000001F79F1BF4A0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={},

In [11]:
vectorstore.similarity_search("Just because we fight without rancor and without selfish object")

[Document(id='133a9e74-edcf-41a9-af32-4969f2c4a9c0', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerent

In [12]:
retriever = vectorstore.as_retriever()

from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F7C9CD32C0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n\nAnswer the following question based only on the provided context: \n< context> \n{context} \n</context>\n\n\n'), additional_kwargs={})])
        

In [13]:
result = retrieval_chain.invoke({
    "input":"Just because we fight without rancor and without selfish object"
})

In [14]:
result['answer']

"Based on the provided context, we can understand the speaker's perspective as follows:\n\n1. **Commitment to Democracy and Political Liberty**: The speaker emphasizes the importance of making the world safe for democracy, rooted in political liberty, without seeking selfish gains.\n\n2. **No Selfish Motivations**: The speaker asserts that there is no desire for conquest, dominion, or material compensation; rather, the goal is to champion the rights of mankind.\n\n3. **Principles of Right and Fair Play**: There is a commitment to conducting operations as belligerents with rightness and fairness, without animosity towards the people of opposing nations.\n\n4. **Friendship with the German People**: The speaker clarifies that the actions are not against the German people but against their irresponsible government, emphasizing a desire for future mutual relations.\n\n5. **Loyalty Among American Citizens**: The speaker acknowledges the loyalty of Americans of German descent, stressing unity